# Output Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine.

Authors: Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [1]:
import pandas as pd
import os

### File paths:

In [17]:
#parent folder
parent_folder = '../02_basic_energy_model/.spinetoolbox/items/exporter/output'
folders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]
if not folders:
    print("No folders found.")
else:
    latest_folder = max(folders, key=lambda x: os.path.getmtime(os.path.join(parent_folder, x)))
    latest_folder_path = os.path.join(parent_folder, latest_folder)
#input folder
latest_folder_path = latest_folder_path.replace('\\', '/')
folder_path = latest_folder_path
folder_path += '/'
#prepared input data
output_file_path = '../02_output_data/01_basic_energy_model_outputs/'

In [3]:
#set name of the relevant files

#input file
output_old = 'Output_exported.xlsx'

## Workflow of the data preparation

### Latest run

In [4]:
df_output_stochastic = pd.read_excel(folder_path + output_old, sheet_name='report__node__stochastic_scenar', index_col=None)

df_output_stochastic.rename(columns={df_output_stochastic.columns[0]: 'Run_datetime'}, inplace=True)
df_output_stochastic.rename(columns={df_output_stochastic.columns[1]: 'Model_datetime'}, inplace=True)

df_output_stochastic['Run_datetime'] = pd.to_datetime(df_output_stochastic['Run_datetime'].str.split('@').str[1].str.strip())

max_datetime = df_output_stochastic['Run_datetime'].max()

### Stochastic

In [10]:
df_output_stochastic = pd.read_excel(folder_path + output_old, sheet_name='report__node__stochastic_scenar', index_col=None)

df_output_stochastic.rename(columns={df_output_stochastic.columns[0]: 'Run_datetime'}, inplace=True)
df_output_stochastic.rename(columns={df_output_stochastic.columns[1]: 'Model_datetime'}, inplace=True)
df_output_stochastic.rename(columns={df_output_stochastic.columns[2]: 'Variable'}, inplace=True)

df_output_stochastic['Run_datetime'] = pd.to_datetime(df_output_stochastic['Run_datetime'].str.split('@').str[1].str.strip())

max_datetime = df_output_stochastic['Run_datetime'].max()

df_output_stochastic = df_output_stochastic[df_output_stochastic['Run_datetime'] == max_datetime]

df_output_stochastic.columns = df_output_stochastic.columns.str.replace('report__node__stochastic_scenario_Report__', '')

df_output_stochastic.head()

,Run_datetime,Model_datetime,Variable,E-Methanol_storage_Kasso__Base,E-Methanol_storage_Kasso__realisation,Hydrogen_storage_Kasso__Base,Hydrogen_storage_Kasso__realisation
423,2024-02-29 09:16:05,2019-01-01T00:00:00,node_state,NaN,0.0,NaN,0.000000
424,2024-02-29 09:16:05,2019-01-01T01:00:00,node_state,NaN,0.0,NaN,0.000000
425,2024-02-29 09:16:05,2019-01-01T02:00:00,node_state,NaN,0.0,NaN,45.204206
426,2024-02-29 09:16:05,2019-01-01T03:00:00,node_state,NaN,25.0,NaN,57.651599
427,2024-02-29 09:16:05,2019-01-01T04:00:00,node_state,NaN,0.0,NaN,135.612618


### unit__node flow

In [11]:
df_output_unit = pd.read_excel(folder_path + output_old, sheet_name='report__unit__node__direction__', index_col=None)

df_output_unit.rename(columns={df_output_unit.columns[0]: 'Run_datetime'}, inplace=True)
df_output_unit.rename(columns={df_output_unit.columns[1]: 'Model_datetime'}, inplace=True)
df_output_unit.rename(columns={df_output_unit.columns[2]: 'Variable'}, inplace=True)

df_output_unit['Run_datetime'] = pd.to_datetime(df_output_unit['Run_datetime'].str.split('@').str[1].str.strip())

max_datetime = df_output_unit['Run_datetime'].max()

df_output_unit = df_output_unit[df_output_unit['Run_datetime'] == max_datetime]

df_output_unit.columns = df_output_unit.columns.str.replace('report__unit__node__direction__stochastic_scenario_Report__', '')

df_output_unit.head()

,Run_datetime,Model_datetime,Variable,CO2_Vaporizer__Carbon_Dioxide__from_node__Base,CO2_Vaporizer__Carbon_Dioxide__from_node__realisation,CO2_Vaporizer__Vaporized_Carbon_Dioxide__to_node__Base,CO2_Vaporizer__Vaporized_Carbon_Dioxide__to_node__realisation,Destilation_Tower__E-Methanol_Kasso__to_node__Base,Destilation_Tower__E-Methanol_Kasso__to_node__realisation,Destilation_Tower__Raw_Methanol__from_node__Base,...,Methanol_Reactor__Hydrogen_Kasso__from_node__Base,Methanol_Reactor__Hydrogen_Kasso__from_node__realisation,Methanol_Reactor__Raw_Methanol__to_node__Base,Methanol_Reactor__Raw_Methanol__to_node__realisation,Methanol_Reactor__Vaporized_Carbon_Dioxide__from_node__Base,Methanol_Reactor__Vaporized_Carbon_Dioxide__from_node__realisation,Methanol_Reactor__Waste_Heat__to_node__Base,Methanol_Reactor__Waste_Heat__to_node__realisation,Solar_Plant_Kasso__Power_Kasso__to_node__Base,Solar_Plant_Kasso__Power_Kasso__to_node__realisation
423,2024-02-29 09:16:05,2019-01-01T00:00:00,unit_flow,NaN,32.756813,NaN,32.756813,NaN,25.0,NaN,...,NaN,32.756813,NaN,31.446541,NaN,32.756813,NaN,7.861635,NaN,0.0
424,2024-02-29 09:16:05,2019-01-01T01:00:00,unit_flow,NaN,32.756813,NaN,32.756813,NaN,25.0,NaN,...,NaN,32.756813,NaN,31.446541,NaN,32.756813,NaN,7.861635,NaN,0.0
425,2024-02-29 09:16:05,2019-01-01T02:00:00,unit_flow,NaN,32.756813,NaN,32.756813,NaN,25.0,NaN,...,NaN,32.756813,NaN,31.446541,NaN,32.756813,NaN,7.861635,NaN,0.0
426,2024-02-29 09:16:05,2019-01-01T03:00:00,unit_flow,NaN,65.513627,NaN,65.513627,NaN,50.0,NaN,...,NaN,65.513627,NaN,62.893082,NaN,65.513627,NaN,15.723270,NaN,0.0
427,2024-02-29 09:16:05,2019-01-01T04:00:00,unit_flow,NaN,0.000000,NaN,0.000000,NaN,0.0,NaN,...,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.0


### Connection__node flow

In [12]:
df_output_connection = pd.read_excel(folder_path + output_old, sheet_name='report__connection__node__direc', index_col=None)

df_output_connection.rename(columns={df_output_connection.columns[0]: 'Run_datetime'}, inplace=True)
df_output_connection.rename(columns={df_output_connection.columns[1]: 'Model_datetime'}, inplace=True)
df_output_connection.rename(columns={df_output_connection.columns[2]: 'Variable'}, inplace=True)

df_output_connection['Run_datetime'] = pd.to_datetime(df_output_connection['Run_datetime'].str.split('@').str[1].str.strip())

max_datetime = df_output_connection['Run_datetime'].max()

df_output_connection = df_output_connection[df_output_connection['Run_datetime'] == max_datetime]

df_output_connection.columns = df_output_connection.columns.str.replace('report__connection__node__direction__stochastic_scenario_Report__', '')

df_output_connection.head()

,Run_datetime,Model_datetime,Variable,pipeline_District_Heating__District_Heating__to_node__Base,pipeline_District_Heating__District_Heating__to_node__realisation,pipeline_District_Heating__Waste_Heat__from_node__Base,pipeline_District_Heating__Waste_Heat__from_node__realisation,pipeline_storage_e-methanol__E-Methanol_Kasso__from_node__Base,pipeline_storage_e-methanol__E-Methanol_Kasso__from_node__realisation,pipeline_storage_e-methanol__E-Methanol_Kasso__to_node__Base,...,pipeline_storage_hydrogen__Hydrogen_storage_Kasso__to_node__Base,pipeline_storage_hydrogen__Hydrogen_storage_Kasso__to_node__realisation,power_line_Wholesale_Kasso__Power_Kasso__from_node__Base,power_line_Wholesale_Kasso__Power_Kasso__from_node__realisation,power_line_Wholesale_Kasso__Power_Kasso__to_node__Base,power_line_Wholesale_Kasso__Power_Kasso__to_node__realisation,power_line_Wholesale_Kasso__Power_Wholesale__from_node__Base,power_line_Wholesale_Kasso__Power_Wholesale__from_node__realisation,power_line_Wholesale_Kasso__Power_Wholesale__to_node__Base,power_line_Wholesale_Kasso__Power_Wholesale__to_node__realisation
423,2024-02-29 09:16:05,2019-01-01T00:00:00,connection_flow,NaN,40.618449,NaN,40.618449,NaN,0.0,NaN,...,NaN,0.000000,NaN,0.0,NaN,21.848795,NaN,21.848795,NaN,0.0
424,2024-02-29 09:16:05,2019-01-01T01:00:00,connection_flow,NaN,40.618449,NaN,40.618449,NaN,0.0,NaN,...,NaN,0.000000,NaN,0.0,NaN,21.848795,NaN,21.848795,NaN,0.0
425,2024-02-29 09:16:05,2019-01-01T02:00:00,connection_flow,NaN,85.822655,NaN,85.822655,NaN,0.0,NaN,...,NaN,45.204206,NaN,0.0,NaN,52.000000,NaN,52.000000,NaN,0.0
426,2024-02-29 09:16:05,2019-01-01T03:00:00,connection_flow,NaN,93.684290,NaN,93.684290,NaN,25.0,NaN,...,NaN,12.447393,NaN,0.0,NaN,52.000000,NaN,52.000000,NaN,0.0
427,2024-02-29 09:16:05,2019-01-01T04:00:00,connection_flow,NaN,77.961019,NaN,77.961019,NaN,0.0,NaN,...,NaN,77.961019,NaN,0.0,NaN,52.000000,NaN,52.000000,NaN,0.0


### Final results

In [18]:
df_output_total_costs = pd.read_excel(folder_path + output_old, sheet_name='report__model', index_col=None)

df_output_total_costs.rename(columns={df_output_total_costs.columns[0]: 'Run_datetime'}, inplace=True)
df_output_total_costs.rename(columns={df_output_total_costs.columns[1]: 'Model_datetime'}, inplace=True)
df_output_total_costs.rename(columns={df_output_total_costs.columns[2]: 'Variable'}, inplace=True)

df_output_total_costs['Run_datetime'] = pd.to_datetime(df_output_total_costs['Run_datetime'].str.split('@').str[1].str.strip())

max_datetime = df_output_total_costs['Run_datetime'].max()

df_output_total_costs = df_output_total_costs[df_output_total_costs['Run_datetime'] == max_datetime]
df_output_total_costs
output_file_path + f"Basic_energy_model_{max_datetime.strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"

'../02_output_data/01_basic_energy_model_outputs/Basic_energy_model_2024-02-29_09-16-05.xlsx'

### Creating one combined excel and export

In [19]:
with pd.ExcelWriter(output_file_path + f"Basic_energy_model_{max_datetime.strftime('%Y-%m-%d_%H-%M-%S')}.xlsx") as writer:
    df_output_total_costs.to_excel(writer, sheet_name='Total costs', index=False)
    df_output_unit.to_excel(writer, sheet_name='Unit__node', index=False)
    df_output_connection.to_excel(writer, sheet_name='Connection__node', index=False)
    df_output_stochastic.to_excel(writer, sheet_name='Stochastic scenario', index=False)